In [1]:
!pip install -q transformers
!pip install numba 

In [2]:
import json, time
import numpy as np
import pandas as pd

In [3]:
!git clone https://github.com/AdarshKumar712/NLP_Shared_Task_2022
%cd NLP_Shared_Task_2022/

Cloning into 'NLP_Shared_Task_2022'...
remote: Enumerating objects: 45988, done.
remote: Counting objects: 100% (8470/8470), done.
remote: Compressing objects: 100% (8294/8294), done.
remote: Total 45988 (delta 179), reused 8466 (delta 176), pack-reused 37518
Receiving objects: 100% (45988/45988), 640.61 MiB | 18.68 MiB/s, done.
Resolving deltas: 100% (1327/1327), done.
Updating files: 100% (48295/48295), done.
/kaggle/working/NLP_Shared_Task_2022


In [4]:
#Loading Train and Test Dataset Files
train_data = pd.read_csv("train_v2.csv")
test_data = pd.read_csv("eval_with_result.csv")

In [5]:
train_data.head()

,url1_lang,url2_lang,pair_id,link1,link2,ia_link1,ia_link2,Geography,Entities,Time,Narrative,Overall,Style,Tone
0,en,en,1484084337_1484110209,https://www.washingtonpost.com/local/virginia-...,https://www.washingtonpost.com/world/the_ameri...,https://web.archive.org/web/www.washingtonpost...,https://web.archive.org/web/www.washingtonpost...,4.0,4.000000,1.000000,4.000000,4.000000,1.666667,2.000000
1,en,en,1484396422_1483924666,https://www.stlucianewsonline.com/guyana-three...,https://www.thestar.com/news/world/europe/2020...,https://web.archive.org/web/www.stlucianewsonl...,https://web.archive.org/web/www.thestar.com/ne...,4.0,4.000000,1.000000,4.000000,3.666667,1.666667,1.333333
2,en,en,1484698254_1483758694,https://www.teaparty.org/trump-brings-in-2020-...,https://www.timesofisrael.com/trump-says-he-do...,https://web.archive.org/web/www.teaparty.org/t...,https://web.archive.org/web/www.timesofisrael....,1.0,2.000000,1.000000,2.333333,2.333333,1.000000,1.333333
3,en,en,1576314516_1576455088,https://gadgets.ndtv.com/apps/news/zomato-uber...,https://gadgets.ndtv.com/internet/news/indian-...,https://web.archive.org/web/gadgets.ndtv.com/a...,https://web.archive.org/web/gadgets.ndtv.com/i...,1.0,2.333333,2.666667,1.666667,2.000000,1.666667,1.666667
4,en,en,1484036253_1483894099,https://news.yahoo.com/india-approves-third-mo...,https://www.channelnewsasia.com/news/asia/indi...,https://web.archive.org/web/news.yahoo.com/ind...,https://web.archive.org/web/www.channelnewsasi...,1.0,1.250000,1.000000,1.250000,1.250000,1.000000,1.000000


# Data Processing

In [6]:
def text_preprocessing(text):
    text = re.sub(r'(@.*?)[\s]', ' ', text)
    text = re.sub(r'&amp;', '&', text)
    text = re.sub(r'\s+', ' ', text).strip()
    return text

In [7]:
import re
from transformers import AutoTokenizer, AutoModel
from sklearn.preprocessing import StandardScaler

# tokenizer = AutoTokenizer.from_pretrained('bert-base-multilingual-cased')
# tokenizer_name = "mbert"

tokenizer = AutoTokenizer.from_pretrained('xlm-roberta-base')
tokenizer_name = "xlm"

def load_data(input_data, input_dir):
  files_present,files_absent = 0,0
  inputs1, inputs2 = list(), list()
  masks1, masks2 = list(), list()
  targets = list()
  for i in range(len(input_data)):
    row = input_data.iloc[i]
    pair = row['pair_id']
    
    y_target = row['Overall']
    # y_scaler = StandardScaler()
    # y_target = y_scaler.transform(y_target.reshape(-1, 1))

    f1, f2 = pair.split("_")
    folder1 = f1[-2:]
    folder2 = f2[-2:]
    try:
      with open(input_dir+"/"+folder1+"/"+f1+".json","r") as f:
        d1 = json.load(f)
      with open(input_dir+"/"+folder2+"/"+f2+".json","r") as f:
        d2 = json.load(f)
    except Exception as E:
      files_absent = files_absent + 1
      continue
    
    files_present = files_present+1

    s1 = text_preprocessing(d1['text'])
    s2 = text_preprocessing(d2['text'])
    encoded_input1 = tokenizer.encode_plus(s1, return_tensors='pt',return_attention_mask=True, pad_to_max_length=True,max_length=256)
    encoded_input2 = tokenizer.encode_plus(s2, return_tensors='pt',return_attention_mask=True, pad_to_max_length=True,max_length=256)

    input_ids1 = encoded_input1['input_ids']
    input_ids2 = encoded_input2['input_ids']
    attention_masks1 = encoded_input1['attention_mask']
    attention_masks2 = encoded_input2['attention_mask']
    
    inputs1.append(input_ids1)
    inputs2.append(input_ids2)
    masks1.append(attention_masks1)
    masks2.append(attention_masks2)
    targets.append(y_target)

  print(f"Files present: {files_present} and Files absent: {files_absent}. % missing = {(files_absent*100)/(files_present+files_absent)}%")
  return inputs1, inputs2, masks1, masks2, targets  

Downloading:   0%|          | 0.00/615 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/4.83M [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/8.68M [00:00<?, ?B/s]

In [8]:
import torch
from torch.utils.data import TensorDataset, DataLoader

def create_dataloaders(inputs1, inputs2, masks1, masks2, targets, batch_size):
    input_tensor1 = torch.tensor([t.numpy() for t in inputs1])
    input_tensor2 = torch.tensor([t.numpy() for t in inputs2])
    mask_tensor1 = torch.tensor([t.numpy() for t in masks1])
    mask_tensor2 = torch.tensor([t.numpy() for t in masks2])
    targets_tensor = torch.tensor(targets)
    dataset = TensorDataset(input_tensor1, input_tensor2, mask_tensor1, mask_tensor2, targets_tensor)
    dataloader = DataLoader(dataset, batch_size=batch_size,shuffle=True)
    return dataloader

In [9]:
batch_size = 16

#Train Set
train_inputs1, train_inputs2, train_masks1, train_masks2, train_targets = load_data(train_data, "articles")
train_dataloader = create_dataloaders(train_inputs1, train_inputs2, train_masks1, train_masks2, train_targets, batch_size)

#Test Set
test_inputs1, test_inputs2, test_masks1, test_masks2, test_targets = load_data(test_data, "eval_data")
test_dataloader = create_dataloaders(test_inputs1, test_inputs2, test_masks1, test_masks2, test_targets, batch_size)

Truncation was not explicitly activated but `max_length` is provided a specific value, please use `truncation=True` to explicitly truncate examples to max length. Defaulting to 'longest_first' truncation strategy. If you encode pairs of sequences (GLUE-style) with the tokenizer you can select this strategy more precisely by providing a specific strategy to `truncation`.
/opt/conda/lib/python3.7/site-packages/transformers/tokenization_utils_base.py:2232: FutureWarning: The `pad_to_max_length` argument is deprecated and will be removed in a future version, use `padding=True` or `padding='longest'` to pad to the longest sequence in the batch, or use `padding='max_length'` to pad to a max length. In this case, you can give a specific length with `max_length` (e.g. `max_length=45`) or leave max_length to None to pad to the maximal input size of the model (e.g. 512 for Bert).
  FutureWarning,


Files present: 4940 and Files absent: 24. % missing = 0.48348106365834004%
Files present: 4659 and Files absent: 243. % missing = 4.957160342717258%


In [10]:
import torch
import torch.nn as nn

if torch.cuda.is_available():       
  device = torch.device("cuda")
  print("Using GPU.")
else:
  print("No GPU available, using the CPU instead.")
  device = torch.device("cpu")

Using GPU.


# Modeling Baselines

In [11]:
#Model Architecure

#Baseline 1: mBERT
class mBERTRegressor(nn.Module):
    
    def __init__(self, drop_rate=0.2, freeze_camembert=False):        
        super(mBERTRegressor, self).__init__()
        D_in, D_out = 1536, 1
        self.mbert = AutoModel.from_pretrained("bert-base-multilingual-cased")   
        self.regressor = nn.Sequential(
            nn.Dropout(drop_rate),
            nn.Linear(D_in, D_out)) 
           
    def forward(self, input_ids1, input_ids2, attention_masks1, attention_masks2):
      outputs1 = self.mbert(input_ids1, attention_masks1)
      outputs2 = self.mbert(input_ids2, attention_masks2)
      #outputs = torch.dot(outputs1[1],outputs2[1])
      #outputs = outputs1[1]+outputs2[1]
      outputs = torch.cat((outputs1[1], outputs2[1]), 1)
    
      # last_hidden_state_cls = outputs[0][:, 0, :]
      last_hidden_state_cls = outputs
      final_outputs = self.regressor(last_hidden_state_cls)
      return final_outputs

      
#Baseline 2: XLM
class XLMRegressor(nn.Module):
    
    def __init__(self, drop_rate=0.2, freeze_camembert=False):        
        super(XLMRegressor, self).__init__()
        D_in, D_out = 1536, 1
        self.xlm = AutoModel.from_pretrained("xlm-roberta-base")
        self.regressor = nn.Sequential(
            nn.Dropout(drop_rate),
            nn.Linear(D_in, D_out)) 
           
    def forward(self, input_ids1, input_ids2, attention_masks1, attention_masks2):
      outputs1 = self.xlm(input_ids1, attention_masks1)
      outputs2 = self.xlm(input_ids2, attention_masks2)
      #outputs = torch.dot(outputs1[1],outputs2[1])
      #outputs = outputs1[1]+outputs2[1]
      outputs = torch.cat((outputs1[1], outputs2[1]), 1)
      # last_hidden_state_cls = outputs[0][:, 0, :]
      last_hidden_state_cls = outputs
      final_outputs = self.regressor(last_hidden_state_cls)
      return final_outputs

In [12]:
# model = mBERTRegressor(drop_rate=0.2).to(device)
model = XLMRegressor(drop_rate=0.2).to(device)

Downloading:   0%|          | 0.00/1.04G [00:00<?, ?B/s]

Some weights of the model checkpoint at xlm-roberta-base were not used when initializing XLMRobertaModel: ['lm_head.bias', 'lm_head.layer_norm.bias', 'lm_head.dense.bias', 'lm_head.dense.weight', 'lm_head.decoder.weight', 'lm_head.layer_norm.weight']
- This IS expected if you are initializing XLMRobertaModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing XLMRobertaModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [13]:
from torch.nn.utils.clip_grad import clip_grad_norm
from transformers import get_linear_schedule_with_warmup
from transformers import AdamW


freeze = False #set to True for 3rd baseline

epochs = 10
total_steps = len(train_dataloader) * epochs
if freeze:
  optimizer = AdamW(model.regressor.parameters(), lr=5e-5, eps=1e-8) 
else:
  optimizer = AdamW(model.parameters(), lr=5e-5, eps=1e-8)    
scheduler = get_linear_schedule_with_warmup(optimizer,num_warmup_steps=0, num_training_steps=total_steps)
loss_function = nn.MSELoss()


def train(model, optimizer, scheduler, loss_function, epochs, train_dataloader, device, clip_value=2):
    for epoch_i in range(epochs):
        print(f"Epoch {epoch_i}")
        print("------------------------------------------------------------")
        best_loss = 1e10

        t0_epoch, t0_batch = time.time(), time.time()
        total_loss, batch_loss, batch_counts = 0, 0, 0
        
        model.train()
        for step, batch in enumerate(train_dataloader): 
            batch_counts +=1
            
            batch_inputs1, batch_inputs2, batch_masks1, batch_masks2, batch_targets = tuple(b.to(device) for b in batch)
            batch_inputs1 = batch_inputs1.squeeze(1)
            batch_inputs2 = batch_inputs2.squeeze(1)
            batch_masks1 = batch_masks1.squeeze(1)
            batch_masks2 = batch_masks2.squeeze(1)
            
            model.zero_grad()
            
            outputs = model(batch_inputs1, batch_inputs2, batch_masks1, batch_masks2)           
            
            loss = loss_function(outputs.squeeze(), batch_targets.float().squeeze())
            batch_loss += loss.item()
            total_loss += loss.item()
            
            loss.backward()
            
            if freeze:
              clip_grad_norm(model.regressor.parameters(), clip_value)
            else:
              clip_grad_norm(model.parameters(), clip_value)
            
            optimizer.step()
            scheduler.step()
            
            if (step % 20 == 0 and step != 0) or (step == len(train_dataloader) - 1):
                # Calculate time elapsed for 20 batches
                time_elapsed = time.time() - t0_batch

                # Print training results
                print(f"{epoch_i + 1:^7} | {step:^7} | {batch_loss / batch_counts:^12.6f} | {'-':^10} | {'-':^9} | {time_elapsed:^9.2f}")

                # Reset batch tracking variables
                batch_loss, batch_counts = 0, 0
                t0_batch = time.time()
              
#             with torch.no_grad():
#                 del outputs
#                 torch.cuda.empty_cache()
        
#             show_gpu(f'{epoch_i}: GPU memory usage after training model:')
            del loss, outputs
            torch.cuda.empty_cache()
            torch.cuda.synchronize()
#             show_gpu(f'{epoch_i}: GPU memory usage after clearing cache:')
        
        avg_train_loss = total_loss / len(train_dataloader)

        print("-"*70)
        print(f"{epoch_i + 1:^7} | {'-':^7} | {avg_train_loss:^12.6f}")
        print();

    return model

In [14]:
# from numba import cuda 
# dev = cuda.get_current_device()
# dev.reset()

In [15]:
# %%time
model = train(model, optimizer, scheduler, loss_function, epochs,train_dataloader, device, clip_value=2)

Epoch 0
------------------------------------------------------------


/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:50: UserWarning: torch.nn.utils.clip_grad_norm is now deprecated in favor of torch.nn.utils.clip_grad_norm_.


   1    |   20    |   2.906416   |     -      |     -     |   26.15  
   1    |   40    |   1.404530   |     -      |     -     |   23.07  
   1    |   60    |   1.401613   |     -      |     -     |   23.06  
   1    |   80    |   1.397575   |     -      |     -     |   23.03  
   1    |   100   |   1.290578   |     -      |     -     |   23.06  
   1    |   120   |   1.300371   |     -      |     -     |   23.06  
   1    |   140   |   1.362193   |     -      |     -     |   23.04  
   1    |   160   |   1.273066   |     -      |     -     |   23.07  
   1    |   180   |   1.353890   |     -      |     -     |   23.06  
   1    |   200   |   1.370127   |     -      |     -     |   23.05  
   1    |   220   |   1.260719   |     -      |     -     |   23.07  
   1    |   240   |   1.261810   |     -      |     -     |   23.05  
   1    |   260   |   1.292420   |     -      |     -     |   23.07  
   1    |   280   |   1.237755   |     -      |     -     |   23.03  
   1    |   300   | 

# Evaluation

In [16]:
from numba import cuda 
# device = cuda.get_current_device()
# device.reset()

In [17]:
def pearson(X,Y):
    print(X,Y)
    xm = torch.mean(X.float())
    ym = torch.mean(Y.float())
    print(xm, ym)
    num = 0.0
    deno1 = 0.0
    deno2 = 0.0
    deno = 0.0
    n = X.shape[0]
    for i in range(n):
        num += (X[i]-xm)*(Y[i]-ym)
        deno1 += (X[i]-xm)*(X[i]-xm)
        deno2 += (Y[i]-ym)*(Y[i]-ym)
    deno = deno1*deno2
    deno = torch.sqrt(deno)
    return (num/deno).item()

from scipy.stats import pearsonr

def evaluate(model, loss_function, test_dataloader, device):
    model.eval()
    test_loss, test_pearson, test_mse = [], [], []
    for batch in test_dataloader:
        batch_inputs1, batch_inputs2, batch_masks1, batch_masks2, batch_targets = tuple(b.to(device) for b in batch)
        batch_inputs1 = batch_inputs1.squeeze(1)
        batch_inputs2 = batch_inputs2.squeeze(1)
        batch_masks1 = batch_masks1.squeeze(1)
        batch_masks2 = batch_masks2.squeeze(1)
        with torch.no_grad():
            outputs = model(batch_inputs1, batch_inputs2, batch_masks1, batch_masks2)   

        loss = loss_function(outputs, batch_targets)
        test_loss.append(loss.item())
        preds = outputs.cpu()
        targs = batch_targets.cpu()
        # print(preds, targs)
        pearson_score = pearson(preds, targs)
        MSE = np.square(np.subtract(preds, targs)).mean()
        test_pearson.append(pearson_score)
        test_mse.append(MSE)
    return test_loss, test_pearson, test_mse

In [18]:
test_loss, test_pearson, test_mse = evaluate(model, loss_function, test_dataloader, device)

/opt/conda/lib/python3.7/site-packages/torch/nn/modules/loss.py:528: UserWarning: Using a target size (torch.Size([16])) that is different to the input size (torch.Size([16, 1])). This will likely lead to incorrect results due to broadcasting. Please ensure they have the same size.
  return F.mse_loss(input, target, reduction=self.reduction)


tensor([[2.0929],
        [2.0202],
        [3.7972],
        [1.6334],
        [3.4155],
        [2.6441],
        [1.6453],
        [2.0940],
        [4.0106],
        [2.2975],
        [3.2330],
        [2.0919],
        [2.1183],
        [3.5080],
        [3.5897],
        [2.0931]]) tensor([1.0000, 1.0000, 3.0000, 1.0000, 1.0000, 4.0000, 3.0000, 1.0000, 3.5000,
        4.0000, 1.0000, 1.0000, 1.0000, 4.0000, 1.0000, 2.0000],
       dtype=torch.float64)
tensor(2.6428) tensor(2.0312)
tensor([[2.0928],
        [2.4836],
        [3.7770],
        [3.6024],
        [2.0930],
        [1.5255],
        [3.6439],
        [2.0931],
        [2.8795],
        [3.6223],
        [2.2961],
        [3.7054],
        [1.5123],
        [2.8212],
        [3.6037],
        [1.7311]]) tensor([1.0000, 3.0000, 4.0000, 1.0000, 4.0000, 1.0000, 2.0000, 1.0000, 1.0000,
        3.0000, 3.5000, 3.0000, 1.0000, 2.0000, 4.0000, 1.0000],
       dtype=torch.float64)
tensor(2.7177) tensor(2.2188)
tensor([[2.0764]

/opt/conda/lib/python3.7/site-packages/torch/nn/modules/loss.py:528: UserWarning: Using a target size (torch.Size([3])) that is different to the input size (torch.Size([3, 1])). This will likely lead to incorrect results due to broadcasting. Please ensure they have the same size.
  return F.mse_loss(input, target, reduction=self.reduction)


In [19]:
print(f"Test Pearson Score is {np.sum(test_pearson)/len(test_pearson)}")
print(f"Test MSE Score is {np.sum(test_mse)/len(test_mse)}")

Test Pearson Score is 0.2879457348488807
Test MSE Score is 2.163341587406707
